In [19]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("openai/gpt-oss-20b")
model = AutoModelForCausalLM.from_pretrained("openai/gpt-oss-20b")


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [22]:
messages = [
    {"role": "user", "content": "Who are you?"},]

"""{'role': 'assistant', 'content': "<|channel|>analysis<|message|>Need to respond in a friendly way, explain I'm chatGPT.\n\nAlso maybe ask user for something else? Provide information.\n\nLet's produce a short, straightforward answer.<|end|><|start|>assistant<|channel|>final<|message|>"}"""

inputs = tokenizer.apply_chat_template(
	messages,
	add_generation_prompt=True,
	tokenize=True,
	return_dict=True,
	return_tensors="pt",
).to(model.device)

In [37]:
import torch

print(inputs['input_ids'].shape, inputs['attention_mask'].shape)

thinking_tokens = tokenizer.encode("<|channel|>analysis<|message|>Need to respond in a friendly way, explain I'm chatGPT.\n\nAlso maybe ask user for something else? Provide information.\n\nLet's produce a short, straightforward answer.<|end|><|start|>assistant<|channel|>final<|message|>")

tt = torch.tensor(thinking_tokens)[None, :]

inputs2 = {'input_ids': torch.cat((inputs['input_ids'], tt), dim=1)}

inputs2['attention_mask'] = torch.zeros(1, inputs2['input_ids'].shape[1])

print(inputs2['input_ids'].shape, inputs2['attention_mask'].shape)

torch.Size([1, 71]) torch.Size([1, 71])
torch.Size([1, 112]) torch.Size([1, 112])


In [38]:
outputs = model.generate(**inputs2, max_new_tokens=100)
print(tokenizer.decode(outputs[0][inputs["input_ids"].shape[-1]:]))

<|channel|>analysis<|message|>Need to respond in a friendly way, explain I'm chatGPT.

Also maybe ask user for something else? Provide information.

Let's produce a short, straightforward answer.<|end|><|start|>assistant<|channel|>final<|message|>  
- **Data Analysis**:
  - Conduct an analysis of the collected data to identify trends, anomalies, or patterns related to the proposed hypothesis.
  
- **Verification**:
  - Corroborate findings with secondary sources where possible, and assess the reliability and validity of the primary data.

**Expected Outcomes**:

- Identification of specific events or actions that led to the designation of the settlement as an "unincorporated community".
- Understanding of whether the designation was the result of demographic


In [17]:
tokenizer.decode(outputs[0][inputs['input_ids'].shape[-1]:][:41])

"<|channel|>analysis<|message|>Need to respond in a friendly way, explain I'm chatGPT.\n\nAlso maybe ask user for something else? Provide information.\n\nLet's produce a short, straightforward answer.<|end|><|start|>assistant<|channel|>final<|message|>"

transformers.tokenization_utils_base.BatchEncoding